In [122]:
import pandas as pd
import numpy as np
import time
import os
from IPython.display import display

In [3]:
import re 
def cleanup(text):

    import string
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [4]:
treinamento1 = pd.read_excel("FelipeNeto_1.xlsx",sheet_name = 'Treinamento', index = False)
treinamento2 = pd.read_excel("FelipeNeto_2.xlsx",sheet_name = 'Treinamento', index = False)
treinamento3 = pd.read_excel("FelipeNeto_3.xlsx",sheet_name = 'Treinamento', index = False)
treinamento4 = pd.read_excel("FelipeNeto_4.xlsx",sheet_name = 'Treinamento', index = False)

lista_treinamentos=[treinamento1,treinamento2,treinamento3,treinamento4]

treinamento=pd.concat(lista_treinamentos,sort=False) #Juntando todas as planilhas

probabilidades=treinamento["Valor"].value_counts(True)#Calculando as frequências relativas de cada categoria

In [5]:
prob_ama = probabilidades[0]
prob_concorda = probabilidades[1]
prob_discorda = probabilidades[2]
prob_odeia = probabilidades[3]
prob_desrel = probabilidades[4]

In [44]:
ama = treinamento[treinamento["Valor"]==0]
ama = ama["Treinamento"].str.cat()

ama = cleanup(ama).split()
set(ama);
serie_ama=pd.Series(ama)

In [47]:
tabela_ama = serie_ama.value_counts(True)
tabela_ama

felipe                  0.051020
neto                    0.050170
o                       0.033163
e                       0.022959
que                     0.020408
de                      0.020408
a                       0.019558
do                      0.017857
eu                      0.016156
é                       0.014456
https                   0.012755
//t                     0.012755
mais                    0.010204
ele                     0.010204
os                      0.009354
não                     0.008503
q                       0.007653
@felipeneto             0.007653
pra                     0.006803
com                     0.005952
se                      0.005952
para                    0.005102
meu                     0.005102
um                      0.005102
ser                     0.005102
mas                     0.004252
por                     0.004252
em                      0.004252
da                      0.004252
neto,                   0.004252
          

In [8]:
concorda = treinamento[treinamento["Valor"]==1]
concorda = concorda["Treinamento"].str.cat()

concorda = cleanup(concorda).split()
set(concorda);
serie_concorda=pd.Series(concorda)

In [124]:
tabela_concorda = serie_concorda.value_counts(True)
tabela_concorda

felipe                             0.039684
o                                  0.033029
neto                               0.032783
de                                 0.026374
que                                0.025635
e                                  0.024649
a                                  0.022430
do                                 0.017501
é                                  0.016515
https                              0.013310
//t                                0.013310
não                                0.011338
um                                 0.009367
com                                0.008381
ele                                0.008134
os                                 0.007641
eu                                 0.007641
pra                                0.007148
neto,                              0.007148
mas                                0.006655
uma                                0.005916
se                                 0.005669
por                             

In [10]:
discorda = treinamento[treinamento["Valor"]==2]
discorda = discorda["Treinamento"].str.cat()

discorda = cleanup(discorda).split()
set(discorda);
serie_discorda=pd.Series(discorda)

In [125]:
tabela_discorda = serie_discorda.value_counts(True)
tabela_discorda

o                             0.034201
felipe                        0.033260
neto                          0.028553
e                             0.027926
que                           0.024788
de                            0.022906
a                             0.021337
é                             0.016316
não                           0.016003
do                            0.012237
um                            0.010668
com                           0.010355
se                            0.009413
ele                           0.008786
eu                            0.007531
os                            0.007217
no                            0.006903
só                            0.005962
em                            0.005962
pra                           0.005962
mas                           0.005962
quem                          0.005648
uma                           0.005020
q                             0.005020
https                         0.005020
//t                      

In [12]:
odeia = treinamento[treinamento["Valor"]==3]
odeia = odeia["Treinamento"].str.cat()

odeia = cleanup(odeia).split()
set(odeia);

serie_odeia=pd.Series(odeia)

In [126]:
tabela_odeia = serie_odeia.value_counts(True)
tabela_odeia

felipe                                   0.038258
neto                                     0.035315
o                                        0.027271
de                                       0.026094
que                                      0.023936
e                                        0.018835
é                                        0.018835
a                                        0.018638
não                                      0.016284
um                                       0.012949
do                                       0.012753
se                                       0.011379
https                                    0.009613
//t                                      0.009613
ele                                      0.009025
da                                       0.008044
em                                       0.007652
eu                                       0.007063
para                                     0.006671
com                                      0.006474


In [127]:
desrelacionado = treinamento[treinamento["Valor"]==4]
desrelacionado = desrelacionado["Treinamento"].str.cat()

desrelacionado = cleanup(desrelacionado).split()
set(desrelacionado);

serie_desrelacionado=pd.Series(desrelacionado)

In [128]:
tabela_desrel = serie_desrelacionado.value_counts(True)
tabela_desrel

felipe                      0.045839
neto                        0.039150
o                           0.038363
que                         0.027936
de                          0.023018
e                           0.020264
a                           0.019477
//t                         0.018886
https                       0.018886
do                          0.014755
é                           0.013378
não                         0.011804
com                         0.010624
um                          0.008853
no                          0.007082
na                          0.006886
da                          0.006886
ele                         0.006886
neto,                       0.005902
eu                          0.005705
só                          0.005312
se                          0.005115
por                         0.005115
tem                         0.004525
os                          0.004525
uma                         0.004525
em                          0.004131
c

In [16]:
def classifica_palavra(frase):
    lista_palavras = frase.split()
    p0 = 1
    p1 = 1
    p2 = 1
    p3 = 1
    p4 = 1
    for word in lista_palavras:
        if word in tabela_ama.index:
            pword0 = tabela_ama[word]
        else:
            pword0 = 1
        if word in tabela_concorda.index:
            pword1 = tabela_concorda[word]
        else:
            pword1 = 1
        if word in tabela_discorda.index:
            pword2 = tabela_discorda[word]
        else:
            pword2 = 1
        if word in tabela_odeia.index:
            pword3 = tabela_odeia[word]
        else:
            pword3 = 1
        if word in tabela_desrel.index:
            pword4 = tabela_desrel[word]
        else:
            pword4 = 1
        p0 *= pword0
        p1 *= pword1
        p2 *= pword2
        p3 *= pword3
        p4 *= pword4
    if p0 == 1:
        p0 -= 1
    if p1 == 1:
        p1 -= 1
    if p2 == 1:
        p2 -= 1
    if p3 == 1:
        p3 -= 1
    if p4 == 1:
        p4 -= 1
    p0 *= prob_ama
    p1 *= prob_concorda
    p2 *= prob_discorda
    p3 *= prob_odeia
    p4 *= prob_desrel
    if (p0 > p1) and (p0 > p2) and (p0 > p3) and (p0 > p4):
        return "Ama"
    elif (p1 > p0) and (p1 > p2) and (p1 > p3) and (p1 > p4):
        return "Concorda"
    elif (p2 > p0) and (p2 > p1) and (p2 > p3) and (p2 > p4):
        return "Discorda"
    elif (p3 > p0) and (p3 > p1) and (p3 > p2) and (p3 > p4):
        return "Odeia"
    elif (p4 > p0) and (p4 > p1) and (p4 > p2) and (p4 > p3):
        return "Desrelacionado"
    
    return "p0: {0} - p1: {1} - p2: {2} - p3: {3} - p4: {4}".format(p0,p1,p2,p3,p4)

In [17]:
experimento = "herói"
print(classifica_palavra(experimento))

Concorda


In [123]:
i = 1
teste_x = pd.read_excel("FelipeNeto.xlsx",sheet_name = 'Teste', index = False)
while True:
    
    lista_teste =[teste_x]
    teste = pd.concat(lista_teste,sort=False) #Juntando todas as planilhas
    
    teste_idx = teste.set_index("Teste")
    lista_tweets = list(teste_idx.index)
    
    
    for tweet in lista_tweets:
        teste_idx.loc[tweet,"Classificação"] = classifica_palavra(tweet)
    
    writer = pd.ExcelWriter('FelipeNetoNaiveBayes_{0}.xlsx'.format(i), engine='xlsxwriter')
    teste_idx.to_excel(writer, sheet_name='Teste')
    
    writer.save()
    
    os.remove("FelipeNeto.xlsx")
    
    i += 1
    time.sleep(10)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'FelipeNeto.xlsx'